In [5]:
from bs4 import BeautifulSoup as bs
import os
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import plotly.express as px
import descartes
import re
from datetime import datetime
import calmap
import geopandas
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import *
import re
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
#import geoplot
#import shapefile as shp
import string

from sklearn.linear_model import LinearRegression, LogisticRegression
import geopandas
import descartes
import geoplot
import shapefile as shp
import warnings; warnings.simplefilter('ignore')

In [108]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
df = spark.read.option("header", "true").csv("data/cat_csv/*.csv")
df.count()
agora_vendors_df = df.select('vendor', 'date').distinct().toPandas()

In [200]:
SR_vendors_df = pd.read_csv("data/total.dat")
evo_vendors_df = pd.read_csv("data/evo_vendors.txt")
c9_vendors_df = pd.read_csv("data/c9_vendors.txt")

In [103]:
def cleanName(s):
    return str(s).translate(string.punctuation).lower()

In [201]:
agora_vendors_df['vendor']    = agora_vendors_df['vendor'].apply(cleanName)
SR_vendors_df['vendor']  = SR_vendors_df['vendor'].apply(cleanName)
evo_vendors_df['vendor'] = evo_vendors_df['vendor'].apply(cleanName)
c9_vendors_df['vendor']  = c9_vendors_df['vendor'].apply(cleanName)

In [202]:
agora_min_df = agora_vendors_df.groupby(by='vendor').min()
agora_max_df = agora_vendors_df.groupby(by='vendor').max()

In [212]:
evo_vendors_df['exact_Evo'] = 1
evo_min_df = evo_vendors_df.groupby(by='vendor').min()
evo_max_df = evo_vendors_df.groupby(by='vendor').max()

In [204]:
SR_vendors_df = pd.DataFrame.drop_duplicates(SR_vendors_df)
c9_vendors_df = pd.DataFrame.drop_duplicates(c9_vendors_df)

In [213]:
SR_vendors_df['exact_SR'] = 1
c9_vendors_df['exact_c9'] = 1

In [229]:
central_df = pd.DataFrame.drop_duplicates(agora_vendors_df.drop(columns=['date']))
central_df = central_df.set_index('vendor').join(agora_min_df).join(agora_max_df, lsuffix='_min', rsuffix='_max')

In [230]:
central_df = central_df.join(evo_min_df, how='left').join(evo_max_df, how='left', lsuffix='_evo_min', rsuffix='_evo_max').rename(columns={'exact_Evo_evo_max' : 'exact_Evo'}).drop(columns=['exact_Evo_evo_min'])

In [260]:
central_df = central_df.join(SR_vendors_df.set_index('vendor'), how='left').join(c9_vendors_df.set_index('vendor'), how='left')

ValueError: columns overlap but no suffix specified: Index(['exact_SR'], dtype='object')

In [262]:
central_df.reset_index(inplace=True)

In [268]:
import hashlib


In [305]:
chaine2 = ""
def hamming_distance(chaine1):
    if (len(chaine1) < 5) & (len(chaine1) != len(chaine2)):
        return 100
    return np.sum(c1 != c2 for c1, c2 in zip(chaine1, chaine2)) + (len(chaine1)-len(chaine2))


def isThereAlmost(name, df):
    chaine2 = name
    res = df['vendor'].apply(hamming_distance)
    
    if len(res[res < 5]) > 0:
        return 1
    else:
        return 0

In [309]:
set(central_df['vendor'].apply(lambda x : isThereAlmost(x, SR_vendors_df)))

{0}

In [311]:
set(central_df['vendor'].apply(lambda x : isThereAlmost(x, evo_min_df.reset_index())))

{0}

In [312]:
set(central_df['vendor'].apply(lambda x : isThereAlmost(x, c9_vendors_df)))

{0}

In [334]:
survivor_all_mkt = central_df[(central_df['date_min'] < '2014-11-07') & (central_df['date_max'] > '2014-11-10')].dropna()
dead_all_mkt = central_df[(central_df['date_min'] < '2014-11-07') & (central_df['date_max'] < '2014-11-10')].dropna()

In [338]:
central_df.fillna(0)

,vendor,date_min,date_max,date_evo_min,date_evo_max,exact_Evo,exact_SR,exact_c9
0,uknextday,2014-04-06,2015-07-07,2014-06-09,2015-01-09,1.0,1.0,0.0
1,chemsdirect,2015-06-20,2015-07-07,0,0,0.0,0.0,0.0
2,chess,2014-02-23,2015-07-07,0,0,0.0,0.0,0.0
3,hulken,2015-06-12,2015-06-26,0,0,0.0,0.0,0.0
4,perfectscans,2014-01-01,2015-06-30,2014-06-09,2015-03-26,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...
3215,swenor,2014-12-20,2014-12-20,0,0,0.0,0.0,0.0
3216,missjulia,2014-03-18,2014-03-18,0,0,0.0,0.0,0.0
3217,tacaocat,2015-07-07,2015-07-07,0,0,0.0,0.0,0.0
3218,madara_theuchiha2,2014-02-28,2014-02-28,0,0,0.0,0.0,0.0


In [361]:
central_df

,vendor,date_min,date_max,date_evo_min,date_evo_max,exact_Evo,exact_SR,exact_c9,two_mkt,three_mkt
0,uknextday,2014-04-06,2015-07-07,2014-06-09,2015-01-09,1.0,1.0,NaN,1.0,1.0
1,chemsdirect,2015-06-20,2015-07-07,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,chess,2014-02-23,2015-07-07,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,hulken,2015-06-12,2015-06-26,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,perfectscans,2014-01-01,2015-06-30,2014-06-09,2015-03-26,1.0,1.0,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
3215,swenor,2014-12-20,2014-12-20,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3216,missjulia,2014-03-18,2014-03-18,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3217,tacaocat,2015-07-07,2015-07-07,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3218,madara_theuchiha2,2014-02-28,2014-02-28,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [383]:
central_df['three_mkt'] = temp['exact_Evo'] * temp['exact_SR'] + temp['exact_Evo'] * temp['exact_c9'] + temp['exact_SR'] *temp['exact_c9']
central_df['three_mkt'] = central_df['three_mkt'].apply(lambda x:  1 if x > 1 else x)

In [440]:
central_df['two_mkt'] = temp['exact_Evo'] +  temp['exact_c9'] + temp['exact_SR']
central_df['two_mkt'] = central_df['two_mkt'].apply(lambda x:  1 if x > 1 else x)

In [441]:
central_df['all_mkt'] = temp['exact_Evo'] * temp['exact_c9'] * temp['exact_SR']


In [442]:
central_df['ag_only'] = 1 - central_df['two_mkt']

In [443]:
central_df.sum()

vendor       uknextdaychemsdirectchesshulkenperfectscansgot...
date_min     2014-04-062015-06-202014-02-232015-06-122014-0...
date_max     2015-07-072015-07-072015-07-072015-06-262015-0...
exact_Evo                                                  986
exact_SR                                                   427
exact_c9                                                   215
two_mkt                                                   1209
three_mkt                                                  374
all_mkt                                                     45
ag_only                                                   2011
dtype: object

In [444]:
agora_here_aft_df = central_df[central_df['date_max'] > '2014-12-10']
agora_here_bef_df = central_df[central_df['date_min'] < '2014-11-06']

## Population that surviced Onymous

In [510]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
agora_surv_df = central_df[(central_df['date_max'] > '2015-01-10') &
                           (central_df['date_min'] < '2014-11-06')]
test = agora_surv_df.sum()
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['2', 'A', '3', '1'],
                             values=[test['two_mkt'] - test['three_mkt'] - test['all_mkt'], test['all_mkt'], test['three_mkt'], test['ag_only']])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Data to 1


## Population arrived after Onymous

In [511]:
agora_aft_df = central_df[(central_df['date_min'] > '2014-11-06')]
test = agora_aft_df.sum()
# Data to 1
# Data to 1
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['2 Shops', '4 Shops', '3 Shops', '1 Shop'],
                             values=[test['two_mkt'] - test['three_mkt'] - test['all_mkt'], test['all_mkt'], test['three_mkt'], test['ag_only']])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Data to 1


## Population that was there before Onymous

In [512]:
agora_bef_df = central_df[(central_df['date_min'] < '2014-11-06')]
test = agora_bef_df.sum()
# Data to 1
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['2 Shops', '4 Shops', '3 Shops', '1 Shop'],
                             values=[test['two_mkt'] - test['three_mkt'] - test['all_mkt'], test['all_mkt'], test['three_mkt'], test['ag_only']])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Data to 1


## Population that died with Onymous

In [513]:
agora_dead_df = central_df[(central_df['date_max'] < '2014-11-06')]
test = agora_dead_df.sum()
# Data to 1
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['2 Shops', '4 Shops', '3 Shops', '1 Shop'],
                             values=[test['two_mkt'] - test['three_mkt'] - test['all_mkt'], test['all_mkt'], test['three_mkt'], test['ag_only']])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Data to 1
